<a href="https://colab.research.google.com/github/rizkyprofs/ML_DL/blob/main/Deep_Learning_Infrared_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Model for Infrared Dataset
Notebook ini mencakup model regresi menggunakan PyTorch dan TensorFlow, serta evaluasi menggunakan RMSE, MSE, dan R².


In [1]:
from google.colab import files
uploaded = files.upload()

Saving Infrared.csv to Infrared (1).csv


In [2]:
!pip install --upgrade torch torchvision torchaudio


Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 78, in main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  Fil

In [3]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load dataset
file_path = "Infrared.csv"
df = pd.read_csv(file_path)
df.head()


,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
0,Male,41-50,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525,36.59
1,Female,31-40,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075,37.19
2,Female,21-30,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850,37.34
3,Female,21-30,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825,37.09
4,Male,18-20,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175,37.04


In [4]:

# Drop non-numeric columns
df = df.select_dtypes(include=[np.number])

# Define features and target
X = df.drop(columns=['aveOralM'])
y = df['aveOralM']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Define PyTorch model
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Initialize model, loss, and optimizer
model = RegressionModel(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train model
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


Epoch 0, Loss: nan
Epoch 10, Loss: nan
Epoch 20, Loss: nan
Epoch 30, Loss: nan
Epoch 40, Loss: nan
Epoch 50, Loss: nan
Epoch 60, Loss: nan
Epoch 70, Loss: nan
Epoch 80, Loss: nan
Epoch 90, Loss: nan


In [7]:
X_train_tensor = torch.nan_to_num(X_train_tensor)
X_test_tensor = torch.nan_to_num(X_test_tensor)

# Predictions
with torch.no_grad():
    y_pred_train = model(X_train_tensor).cpu().numpy()
    y_pred_test = model(X_test_tensor).cpu().numpy()

# Pastikan tidak ada NaN dalam prediksi
y_pred_train = np.nan_to_num(y_pred_train)
y_pred_test = np.nan_to_num(y_pred_test)

# Evaluasi Model
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
mse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f'PyTorch Model - RMSE: {rmse}, MSE: {mse}, R2: {r2}')

PyTorch Model - RMSE: 37.03647032741816, MSE: 1371.7001343137256, R2: -6513.15786634289


In [8]:

# TensorFlow Model
tf_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile model
tf_model.compile(optimizer='adam', loss='mse')

# Train model
tf_model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: nan
Epoch 2/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan
Epoch 3/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan
Epoch 4/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan
Epoch 5/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan
Epoch 6/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan
Epoch 7/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: nan
Epoch 8/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan
Epoch 9/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan
Epoch 10/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan
Epoch 11/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan
Epoch 12/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan
Epoch 13/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: nan
Epoch 14/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: nan
Epoch 15/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan
Epoch 16/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/st

In [10]:
# Pastikan tidak ada NaN dalam input data sebelum diprediksi
X_test = np.nan_to_num(X_test)

# Predictions
y_pred_test_tf = tf_model.predict(X_test)

# Pastikan tidak ada NaN dalam hasil prediksi
y_pred_test_tf = np.nan_to_num(y_pred_test_tf)

# Evaluation
rmse_tf = np.sqrt(mean_squared_error(y_test, y_pred_test_tf))
mse_tf = mean_squared_error(y_test, y_pred_test_tf)
r2_tf = r2_score(y_test, y_pred_test_tf)

print(f'TensorFlow Model - RMSE: {rmse_tf}, MSE: {mse_tf}, R2: {r2_tf}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
TensorFlow Model - RMSE: 37.03647032741816, MSE: 1371.7001343137256, R2: -6513.15786634289



## Penjelasan Matematika

### 1. Mean Squared Error (MSE)
\[ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 \]
Di mana:
- \( y_i \) adalah nilai aktual,
- \( \hat{y}_i \) adalah nilai prediksi,
- \( n \) adalah jumlah sampel.

MSE mengukur rata-rata kesalahan kuadrat antara prediksi dan nilai aktual.

### 2. Root Mean Squared Error (RMSE)
\[ RMSE = \sqrt{MSE} \]
RMSE memberikan gambaran seberapa besar kesalahan rata-rata dalam unit yang sama dengan target variabel.

### 3. R-Squared (R²)
\[ R^2 = 1 - \frac{\sum (y_i - \hat{y}_i)^2}{\sum (y_i - \bar{y})^2} \]
Di mana \( \bar{y} \) adalah rata-rata nilai aktual.

R² mengukur seberapa baik model menjelaskan variasi dalam data.
